In [1]:
!pip install langchain llama_index pandas plotly

In [2]:
import os
from collections import defaultdict
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document, PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core import Settings
Settings.chunk_size = 1024
Settings.chunk_overlap = 100

In [3]:
# Set API keys and configurations
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"# your_openai_api_key

In [4]:
# Initialize embeddings and LLM
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
llm = OpenAI(temperature=0, model_name="text-davinci-003")

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [5]:
mpath = './'
# Load log lines from text file
with open(mpath + 'demofile1.txt', 'r') as file:
    log_lines = file.readlines()
    
log_lines = [j.rstrip('\n') for j in log_lines]
log_lines = [j for j in log_lines if j]

In [6]:
# Convert log lines to LlamaIndex documents
documents = [Document(text=line.strip(), metadata={'key':i}) for i, line in enumerate(log_lines)]

# Initialize LlamaIndex
index = VectorStoreIndex(documents, embed_fn=embedding_model.embed_query)

In [7]:
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query("describe any issue present")
print(response)

There is a potential security vulnerability detected in module_4 and a critical issue of low disk space in module_1.


In [8]:
retriever = index.as_retriever(similarity_top_k=10)
# please use any to see if this can match or find issues in available logs
"""
Firmware corruption
Overheating hardware
Misconfigured routing protocols
VPN tunnel failures
Port forwarding errors
VLAN misconfiguration
Spanning tree loops
Port security violations
Faulty switch fabric
Insufficient PoE power
hardware
vulnerability
"""
nodes = retriever.retrieve("hardware")
nodes = [j for j in nodes if j.score>0.7]
nodes = sorted(nodes, key=lambda x:x.score, reverse=True)[:3]
for n in nodes:
    print(f'Log Line      : {n.text}')
    print(f'Matching Score: {n.score}')

Log Line      : 2024-06-02T13:10:20Z [WARNING] High temperature alert in chassis, current temperature: 60°C.
Matching Score: 0.7616678536254086
Log Line      : Jun 3 10:15:09 Router1 %QOS-5-QOS_FLOW_MONITORING: 80% of bandwidth utilization reached on GigabitEthernet0/0
Matching Score: 0.7525510321214063
Log Line      : Jun 3 10:15:00 Router1 %LINK-5-CHANGED: Interface GigabitEthernet0/0, changed state to administratively down
Matching Score: 0.7514924549799252


In [9]:
with open(mpath + 'signatures.txt', 'r') as file:
    signatures = file.readlines()
signatures = [j.replace('\n', '') for j in signatures]
signatures = ['####' if j=='' else j for j in signatures]
signatures = ' \n '.join(signatures)
signatures = signatures.split('####')
signatures = [j for j in signatures if j not in ('', ' \n ', '####')]

In [10]:
# Define a search function
def search_logs(query, top_k=5, thre = 0.7):
    retriever = index.as_retriever(similarity_top_k=top_k*3)
    nodes = retriever.retrieve(query) #vulnerability
    nodes = [j for j in nodes if j.score>thre]
    nodes = sorted(nodes, key=lambda x:x.score, reverse=True)[:top_k]
    #nodes = [j.text for j in nodes]

    return nodes

In [11]:
# Function to generate a report using Langchain
def generate_report(query, retrieved_docs):
    prompt_template = """
    Based on the following query and logs:
    Query: {query}
    Logs: {logs}
    Generate a detailed report, describe the problem and recommend a solution. 
    please remove those logs that are not relevant to query
    """
    #prompt = PromptTemplate.from_template(prompt_template)
    #message = prompt.format(query=query, logs="\n"+"\n".join([doc.text for doc in retrieved_docs]))
    prompt = PromptTemplate.from_template(prompt_template)
    llm = OpenAI(temperature=0,model_name="gpt-3.5-turbo-instruct")
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run({"query": query, "logs": " ".join([doc.text for doc in retrieved_docs])})
    return response

In [12]:
# Example usage for search and reporting
query = "Error in module 3"
retrieved_logs = search_logs(query, top_k=3, thre = 0.7)
report = generate_report(query, retrieved_logs)
log_text = '\n'.join([j.text for j in retrieved_logs])
print(f"Relevant Logs: \n{log_text}\n\n{report}")

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Relevant Logs: 
2024-06-01T12:00:00Z ERROR module_3 Unexpected null pointer exception occurred.
2024-06-01T12:15:00Z ERROR module_1 Disk space critically low.
2024-06-01T12:05:00Z WARN module_4 Potential security vulnerability detected.


Report:

Problem:
The query "Error in module 3" indicates that there is an error occurring in module 3. The logs provided show three different errors, but only one of them is related to module 3. The other two errors are not relevant to the query and should be removed.

Solution:
To address the issue, the logs that are not relevant to the query should be removed. This will help to narrow down the search and focus on the specific error in module 3. Once the irrelevant logs are removed, the remaining logs can be analyzed to identify the root cause of the error in module 3.

Detailed Report:
The logs provided show three different errors occurring at different times. The first error, "Unexpected null pointer exception occurred" is related to module 3 and 

In [13]:
# Find logs related to a specific problem
problem_description = "Null pointer exception"
query = f"Find logs related to the following problem: {problem_description}"
related_logs = search_logs(query, top_k=5, thre = 0.7)
related_logs_report = generate_report(problem_description, related_logs)
print("Related Logs Report:", related_logs_report)

Related Logs Report: 
Problem:
The query "Null pointer exception" is causing a null pointer exception error in the system. This error is causing critical issues such as unexpected router reboots, memory allocation errors, and logging to host failures.

Logs:
2024-06-01T12:00:00Z ERROR module_3 Unexpected null pointer exception occurred.
2024-06-02T13:30:40Z [CRITICAL] Router rebooted unexpectedly, last log messages may indicate cause.
2024-06-02T13:20:30Z [ERROR] Memory allocation error on process 'httpd', device may be under a DoS attack.

Detailed Report:
Based on the logs provided, it can be seen that the null pointer exception error occurred on 2024-06-01T12:00:00Z in module_3. This error is causing critical issues such as unexpected router reboots and memory allocation errors. The error is also affecting the logging system, as seen in the logs on Jun 3 10:15:00, where the logging to host 192.168.1.3 stopped.

The null pointer exception error is a common error in programming and ca

In [14]:
# Anomaly Detection

In [15]:
# searching exhaustively for log anomalies
# this can be optimized for performance by storing embeddings for signatures from past anomalies
anomalies = []
for sig in signatures:
    sig = sig.strip()
    nodes = search_logs(sig, top_k=3, thre = 0.7)
    anomalies.append([sig.split(' \n ')[0],  nodes])

In [16]:
anomalies[0]

['Hardware and Configuration Errors',
 [NodeWithScore(node=Document(id_='a8a9bb6d-8d00-4af5-adf6-cccbe4575a8f', embedding=None, metadata={'key': 60}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Jun 3 10:15:00 Router1 %LINK-5-CHANGED: Interface GigabitEthernet0/1, changed state to administratively down', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.8233572312546201),
  NodeWithScore(node=Document(id_='4828ca96-e706-4cfb-8f56-dad2797f9659', embedding=None, metadata={'key': 38}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Jun 3 10:15:00 Router1 %LINK-5-CHANGED: Interface GigabitEthernet0/0, changed state to administratively down', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.82314743161

In [17]:
# Anomalies has matched to multiple issues, need disambiguation, you can write complex disambiguation and replace this
common_nodes = defaultdict(list)
for ar in anomalies:
    for i in ar[1]:
        common_nodes[i.node.metadata['key']].append([ar[0], i.score])

In [18]:
for i in common_nodes[19]:
    print(i)

['Network Latency Issues', 0.8201082532191438]
['Packet Loss Issues', 0.8129348685243899]
['Broadcast Storm Issues', 0.7936497638372606]
['Traffic Overload', 0.8323136167370643]
['Overutilization of Resources', 0.842460503126184]
['Underutilization of Resources', 0.8267748506155755]


In [19]:
print(documents[19].text)

Jun 3 10:15:09 Router1 %QOS-5-QOS_FLOW_MONITORING: 80% of bandwidth utilization reached on GigabitEthernet0/0


In [20]:
nodes_best_match = {k:max(v, key=lambda x:x[1]) for k,v in common_nodes.items()}

In [21]:
nodes_best_match[19]

['Overutilization of Resources', 0.842460503126184]

In [22]:
anomalies_selected = []
for ar in anomalies:
    selected_recs = []
    for i in ar[1]:
        rec_no = i.metadata['key']
        if rec_no in nodes_best_match:
            cat = nodes_best_match[rec_no][0]
            if ar[0] != cat:
                continue
        selected_recs.append(i)
    anomalies_selected.append([ar[0], selected_recs])

anomalies_selected = [i for i in anomalies_selected if i[1]]

In [23]:
# Plot anomalies
from sklearn.manifold import TSNE
import pandas as pd
import plotly.io as pio
import plotly.express as px
pio.renderers.default='browser'

In [24]:
df = pd.DataFrame()

for a_s in anomalies_selected:
    for i in a_s[1]:
        dfp = pd.DataFrame([embedding_model.embed_query(i.node.text)], [a_s[0]])
        df = pd.concat([df, dfp])

In [25]:
 # We want to get TSNE embedding with 2 dimensions
n_components = 2
tsne = TSNE(n_components, perplexity=df.shape[0]//2)
tsne_result = tsne.fit_transform(df.values)
tsne_result.shape
# (25, 2)
# Two dimensions for each anomaly

(25, 2)

In [26]:
#Please check your browser for the plot, hovering over scatter dots should reveal information
fig = px.scatter(x=tsne_result[:, 0], 
                 y=tsne_result[:, 1], 
                 color=df.index
                 )
fig.update_traces(marker={'size': 50})
fig.update_layout(
    title="t-SNE visualization of Anomaly Detection",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

In [27]:
# Generate report for each anomaly
anomalies_reports = []
for ano in anomalies_selected:
    anomaly_report = generate_report(ano[0], ano[1])
    anomalies_reports.append([ ano[0], ano[1], anomaly_report])

In [28]:
for ano in anomalies_reports:
    print(ano[0])
    log_text = '\n'.join([j.text for j in ano[1]])
    print(f"Anomaly Report: \n{log_text}\n\n{ano[2]}")
    print('\n-----------------\n')
    #print(input())

Hardware and Configuration Errors
Anomaly Report: 
Jun 3 10:15:00 Router1 %LINK-5-CHANGED: Interface GigabitEthernet0/1, changed state to administratively down
Jun 3 10:15:00 Router1 %LINK-5-CHANGED: Interface GigabitEthernet0/0, changed state to administratively down
Jun 3 10:15:00 Router1 %LINK-5-CHANGED: Interface GigabitEthernet0/0, changed state to administratively down


Report:

Problem:
The logs show that there are multiple interface errors on Router1, specifically on the GigabitEthernet0/0 and GigabitEthernet0/1 interfaces. These errors are causing the interfaces to be administratively down, which can result in network connectivity issues and potential downtime.

Possible Causes:
1. Hardware failure: The errors could be caused by a hardware failure on the interfaces, such as a faulty cable or port.
2. Configuration errors: The errors could be a result of incorrect configuration settings on the interfaces.
3. Software bugs: The errors could be caused by software bugs or glitche